<a href="https://colab.research.google.com/github/TitasDas/PerigonAI_Interview/blob/main/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem 1: Python programming, data processing.**



In this problem we want to generate pseudo-random data that has certain desired statistical properties. This can be useful for demo, research or testing purposes.

First, let’s generate these “desired statistical properties”.

•              Generate a random 6x6 correlation matrix rho.

•              Regularization: write a test checking that rho is a valid correlation matrix, and if not - find the nearest valid one.

Now, let’s generate the data that would have these properties.

•              Generate a set of 6 random variables (put them in a matrix 1000x6, the distribution doesn’t matter, but should be continuous), distributed between 0 and 1 with correlation defined by rho.



Slightly different, but related problem.

•              Apply PCA to reduce the dimensionality to 5.

•              Let the output variable y = round(x6).

•              Build a couple of classifiers of your choice to predict y from {x1, x2, …, x5}.

•              Compare their performance.

First, we generate a random correlation matrix rho as per the size definitions and make sure it is a valid correlation matrix given some assumptions.

In [2]:
# Generating a random 6*6 correlation matrix 'rho'

import numpy as np
A = np.random.randn(6, 6)

# Symmetrize the randomly generated matrix
A_symm = (A + A.T) / 2

# Ensure the diagonal is 1
for i in range(6):
    A_symm[i, i] = 1

# Adjust the matrix to make it a valid correlation matrix
eigvals, eigvecs = np.linalg.eigh(A_symm)
D = np.diag(np.maximum(eigvals, 0))
rho = eigvecs.dot(D).dot(eigvecs.T)

# Ensure the diagonal is 1 again
for i in range(6):
    rho[i, i] = 1

print(rho)

[[ 1.         -0.73877154 -0.047035    0.18865237 -0.1541974  -0.36650824]
 [-0.73877154  1.         -0.52315461 -1.0516195  -0.03403461  1.07231673]
 [-0.047035   -0.52315461  1.          0.7230642  -0.57358697  0.30873532]
 [ 0.18865237 -1.0516195   0.7230642   1.          0.83295554 -0.77829933]
 [-0.1541974  -0.03403461 -0.57358697  0.83295554  1.         -0.49384768]
 [-0.36650824  1.07231673  0.30873532 -0.77829933 -0.49384768  1.        ]]


While generating these kind of matrices we may encounter cases wherein the matrix generated is not "positive semi-definite". In the above procedure we have tried to fix that by zeroing out any neg eigenvalues but this doesnt always generate a matrix with correlations between -1 and 1. We can either repeat the above process or use a **"nearcorr"** algo to produce valid correlation matrices.

Nearcorr algorithm -
1. Start with initial matrix Y.
2. Project Y onto space of matrices with ones on the diagonal to get R.
3. Project R onto space of positive semidefinite matrices to get Y'.
4. If Y' is sufficiently close to Y, then stop; otherwise, set Y = Y' and repeat.

For the positive semidefinite projection, use the spectral decomposition and set any negative eigen values to zero.

In [3]:
import numpy as np

def nearcorr(A, tol=1e-5, max_iter=100):
    n, m = A.shape
    if n != m:
        raise ValueError("Input matrix must be square.")

    # Initialize
    Y = A.copy()
    I = np.eye(n)
    iteration = 0
    convergence = False

    while not convergence and iteration < max_iter:
        iteration += 1

        # Project onto the set of matrices with ones on the diagonal
        R = Y - np.diag(np.diagonal(Y)) + I

        # Project onto the set of positive semidefinite matrices
        eigvals, eigvecs = np.linalg.eigh(R)
        Q = np.dot(eigvecs, np.diag(np.maximum(eigvals, 0)))
        Y_next = np.dot(Q, eigvecs.T)

        # Check convergence
        if np.linalg.norm(Y - Y_next, 'fro') < tol:
            convergence = True

        Y = Y_next.copy()

    return Y

# Create a random symmetric matrix
A = np.random.randn(6, 6)
A_symm = (A + A.T) / 2

# Adjust its diagonal to have ones
np.fill_diagonal(A_symm, 1)

# Apply the nearcorr algorithm
correlation_matrix = nearcorr(A_symm)
print(correlation_matrix)

[[ 1.0000007  -0.20376603 -0.65573185  0.59972383 -0.44980238 -0.35581176]
 [-0.20376603  1.00000156  0.4015529   0.52692701  0.05677801  0.53370809]
 [-0.65573185  0.4015529   1.00000117 -0.17946673  0.72123857 -0.07198055]
 [ 0.59972383  0.52692701 -0.17946673  1.00000159 -0.55108977 -0.16123572]
 [-0.44980238  0.05677801  0.72123857 -0.55108977  1.00000052  0.09494295]
 [-0.35581176  0.53370809 -0.07198055 -0.16123572  0.09494295  1.00000072]]


Next we check if rho is a valid correlation matrix using the following conditions -

1. Its a square.
2. Its diagonal consists of one
3. Its symmetric
4. Its positive semidefinite

In the code below, you can test any matrix rho by passing it to adjust_to_correlation_matrix(). If rho is already a valid correlation matrix, the function will return it unchanged. If not, it will adjust rho to make it a valid correlation matrix using the nearcorr method.

In [4]:
import numpy as np

def is_valid_correlation_matrix(rho):
    # Check if square
    if rho.shape[0] != rho.shape[1]:
        return False

    # Check if diagonal is made of ones
    if not np.allclose(np.diag(rho), 1):
        return False

    # Check if symmetric
    if not np.allclose(rho, rho.T):
        return False

    # Check if positive semidefinite
    eigvals = np.linalg.eigvalsh(rho)
    if np.any(eigvals < 0):
        return False

    return True

def adjust_to_correlation_matrix(rho):
    if is_valid_correlation_matrix(rho):
        return rho
    else:
        return nearcorr(rho)

# Create a random symmetric matrix (just for testing purposes)
A = np.random.randn(6, 6)
A_symm = (A + A.T) / 2

# Adjust its diagonal to have ones
np.fill_diagonal(A_symm, 1)

# Adjust the matrix to be a valid correlation matrix
rho = adjust_to_correlation_matrix(A_symm)
print(rho)


[[ 1.00000394 -0.66080499 -0.12604101  0.94145927 -0.28860038 -0.17627801]
 [-0.66080499  1.00000235  0.14030468 -0.68219438 -0.34365582 -0.42587107]
 [-0.12604101  0.14030468  1.00000011 -0.3389069   0.14204464 -0.21709031]
 [ 0.94145927 -0.68219438 -0.3389069   1.00000489 -0.40190187  0.08996778]
 [-0.28860038 -0.34365582  0.14204464 -0.40190187  1.00000098  0.1375898 ]
 [-0.17627801 -0.42587107 -0.21709031  0.08996778  0.1375898   1.00000059]]


Next, we generate the data that would have these properties :

• Generate a set of 6 random variables (put them in a matrix 1000x6, the distribution doesn’t matter, but should be continuous), distributed between 0 and 1 with correlation defined by rho.

To generate this we use the Cholesky decomposition of the desired correlation matrix -
1. Start with some uncorrelated random data.
2. Use the Cholesky decomposition of the desired correlation matrix to transform the data so that it has the desired correlations

In [7]:

# Function to generate a nearest valid correlation matrix
def nearcorr(A, tol=1e-5, max_iter=100):
    n, m = A.shape
    if n != m:
        raise ValueError("Input matrix must be square.")

    Y = A.copy()
    I = np.eye(n)
    iteration = 0
    convergence = False

    while not convergence and iteration < max_iter:
        iteration += 1
        R = Y - np.diag(np.diagonal(Y)) + I
        eigvals, eigvecs = np.linalg.eigh(R)
        Q = np.dot(eigvecs, np.diag(np.maximum(eigvals, 0)))
        Y_next = np.dot(Q, eigvecs.T)

        if np.linalg.norm(Y - Y_next, 'fro') < tol:
            convergence = True

        Y = Y_next.copy()

    return Y

# Function to check if given matrix is a valid correlation matrix
def is_valid_correlation_matrix(rho):
    if rho.shape[0] != rho.shape[1]:
        return False
    if not np.allclose(np.diag(rho), 1):
        return False
    if not np.allclose(rho, rho.T):
        return False
    eigvals = np.linalg.eigvalsh(rho)
    if np.any(eigvals < 0):
        return False
    return True

# Function to adjust given matrix to a valid correlation matrix
def adjust_to_correlation_matrix(rho):
    if is_valid_correlation_matrix(rho):
        return rho
    else:
        return nearcorr(rho)

# Function to generate data with given correlation matrix
def generate_data_with_correlation(num_samples, rho):
    chol = np.linalg.cholesky(rho)
    z = np.random.rand(num_samples, 6)
    x = z @ chol.T
    x_standard = (x - x.mean(axis=0)) / x.std(axis=0)
    x_rescaled = (x_standard - x_standard.min(axis=0)) / (x_standard.max(axis=0) - x_standard.min(axis=0))
    return x_rescaled

# Create a random symmetric matrix
A = np.random.randn(6, 6)
A_symm = (A + A.T) / 2
np.fill_diagonal(A_symm, 1)
rho = adjust_to_correlation_matrix(A_symm)

# Generate the data with the desired correlation matrix
data = generate_data_with_correlation(1000, rho)
print(data[:5, :])  # Print first 5 rows to inspect


LinAlgError: ignored

Our correlation matrix adjustment method ensures the matrix is positive semidefinite, but this isn't enough to guarantee it's positive definite. A positive semidefinite matrix may still have zero eigenvalues, causing the Cholesky decomposition to fail.

There are several ways to address this:

1. Perturb the matrix slightly by adding a small value to the diagonal, ensuring it becomes positive definite.

2. Use another decomposition method which doesn't require the matrix to be positive definite.

here we try the first method and retry generating the data using the slightly modified 'generate_data_with_correlation' function that perturbs the correlation matrix if needed. This should now be resilient to the error we encountered earlier. It will adjust the correlation matrix, if needed, to ensure it's positive definite before trying to decompose it.

In [10]:
import numpy as np

# Function to generate a nearest valid correlation matrix
def nearcorr(A, tol=1e-5, max_iter=100):
    n, m = A.shape
    if n != m:
        raise ValueError("Input matrix must be square.")

    Y = A.copy()
    I = np.eye(n)
    iteration = 0
    convergence = False

    while not convergence and iteration < max_iter:
        iteration += 1
        R = Y - np.diag(np.diagonal(Y)) + I
        eigvals, eigvecs = np.linalg.eigh(R)
        Q = np.dot(eigvecs, np.diag(np.maximum(eigvals, 0)))
        Y_next = np.dot(Q, eigvecs.T)

        if np.linalg.norm(Y - Y_next, 'fro') < tol:
            convergence = True

        Y = Y_next.copy()

    return Y

def is_valid_correlation_matrix(rho):
    if rho.shape[0] != rho.shape[1]:
        return False
    if not np.allclose(np.diag(rho), 1):
        return False
    if not np.allclose(rho, rho.T):
        return False
    eigvals = np.linalg.eigvalsh(rho)
    if np.any(eigvals < 0):
        return False
    return True

def adjust_to_correlation_matrix(rho):
    if is_valid_correlation_matrix(rho):
        return rho
    else:
        return nearcorr(rho)

def generate_data_with_correlation(num_samples, rho):
    min_eigval = np.min(np.real(np.linalg.eigvals(rho)))
    if min_eigval <= 0:
        rho -= 10*min_eigval * np.eye(rho.shape[0])

    chol = np.linalg.cholesky(rho)
    z = np.random.rand(num_samples, 6)
    x = z @ chol.T
    x_standard = (x - x.mean(axis=0)) / x.std(axis=0)
    x_rescaled = (x_standard - x_standard.min(axis=0)) / (x_standard.max(axis=0) - x_standard.min(axis=0))

    return x_rescaled

# Create a random symmetric matrix
A = np.random.randn(6, 6)
A_symm = (A + A.T) / 2
np.fill_diagonal(A_symm, 1)
rho = adjust_to_correlation_matrix(A_symm)

# Generate the data with the desired correlation matrix
data = generate_data_with_correlation(1000, rho)
print(data[:5, :])  # Print first 5 rows to inspect


[[0.22631914 0.67932848 0.34280903 0.62388448 0.65866416 0.592494  ]
 [0.65988948 0.46903114 0.68832468 0.42901403 0.43466355 0.53611544]
 [0.69183439 0.29144469 0.27235356 0.5191502  0.24665957 0.19213508]
 [0.50672786 0.13011309 0.27376519 0.35911791 0.40187082 0.17499456]
 [0.13599032 0.11894406 0.37158198 0.20455189 0.74808051 0.35500651]]


Next, we apply PCA to reduce the dimensionality from 6 to 5.

In [11]:
from sklearn.decomposition import PCA

# Apply PCA and reduce the dimensionality to 5
pca = PCA(n_components=5)
data_pca = pca.fit_transform(data)

print(data_pca[:5, :])  # Print the first 5 rows to inspect


[[ 2.29260255e-01  2.38718088e-01  2.52424458e-01  3.36512685e-09
   4.85350625e-09]
 [-1.49254980e-01  2.45664399e-03 -2.41362277e-01 -1.03342644e-08
  -2.29525666e-11]
 [-1.87776055e-01 -4.83368627e-01  1.19015083e-01 -1.48916521e-08
  -4.22719375e-09]
 [ 1.19737469e-01 -5.53056120e-01  1.47651341e-02  3.67999227e-09
  -2.85177999e-08]
 [ 6.08969834e-01 -2.94114755e-01 -1.15220864e-01 -2.80014018e-10
   1.60570824e-08]]


Next we round the 6th column of data.

In [13]:
y = np.round(data[:, 5])

# Print the first 10 values to inspect
print(y[:10])


[1. 1. 0. 0. 0. 1. 0. 1. 1. 0.]


The two classifiers we build are a Random Forest classifier and a Logistic regression classifier. We compare their performance using accuracy as the metric.

Here are the steps -
1. Split the data into training and testing sets.
2. Train both classifiers on the training set.
3. Predict on the test set.
4. Compare their accuracy.

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data_pca, y, test_size=0.2, random_state=42)

# Initialize classifiers
rf = RandomForestClassifier(n_estimators=100, random_state=42)
lr = LogisticRegression(max_iter=1000, random_state=42)

# Train classifiers
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

# Predict using the trained classifiers
y_pred_rf = rf.predict(X_test)
y_pred_lr = lr.predict(X_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print(f"Random Forest Accuracy: {accuracy_rf:.2f}")
print(f"Logistic Regression Accuracy: {accuracy_lr:.2f}")

Random Forest Accuracy: 0.98
Logistic Regression Accuracy: 0.99
